In [1]:
# pick a hmm classifier and run through features,
# two different ways to analyse accuracy, confusion matrix per sample or combined model

In [17]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [19]:
import src.ads_davidjames9610.ads as ads
import src.ads_davidjames9610.useful as useful
import numpy as np
import os
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
from importlib import reload
import src.misc_davidjames9610.fe_methods as fe
import src.misc_davidjames9610.proc_methods as pm
from hmmlearn.hmm import GaussianHMM, GMMHMM
from src.classifiers_davidjames9610.test_a.config import config_location
import importlib
config = importlib.import_module(config_location)
reload(config)

<module 'src.classifiers_davidjames9610.test_a.ivector_speech_data.config' from '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/ivector_speech_data/config.py'>

In [27]:
import pickle
sls = pickle.load(open(config.samples_labels,'rb'))
features = pickle.load(open(config.features,'rb'))

In [21]:
print([key for key in features])

['lp', 'lpGaussNoise_SNR60', 'lpGaussNoise_SNR40', 'lpGaussNoise_SNR20', 'lpGaussNoise_SNR10', 'mfcc', 'mfccGaussNoise_SNR60', 'mfccGaussNoise_SNR40', 'mfccGaussNoise_SNR20', 'mfccGaussNoise_SNR10']


In [22]:
# config
number_of_iterations = 1
n_components = 8

In [36]:
classifiers = [
    {
    'type': 'GaussianHMM', # 'fhmm', 'GMMHMM',
    'features': ['lp', 'lpGaussNoise_SNR60', 'lpGaussNoise_SNR40', 'lpGaussNoise_SNR20', 'lpGaussNoise_SNR10'],
    'trained_classifiers': {}
    },
    {
    'type': 'fhmm', # 'fhmm', 'GMMHMM',
    'features': ['lpGaussNoise_SNR60', 'lpGaussNoise_SNR40', 'lpGaussNoise_SNR20', 'lpGaussNoise_SNR10'],
    'trained_classifiers': {}
    },
    {
    'type': 'GMMHMM', # 'fhmm', 'GMMHMM',
    'features': ['lp', 'lpGaussNoise_SNR60', 'lpGaussNoise_SNR40', 'lpGaussNoise_SNR20', 'lpGaussNoise_SNR10'],
    'trained_classifiers': {}
    },
]

In [37]:
import src.fhmm_davidjames9610.fhmm as fhmm
reload(fhmm)
# classifiers = {} # one for feature
cv_index = 0 # TODO expand later

for classifier in classifiers:

    all_features_for_classifier = classifier['features']
    classifier_type = classifier['type']

    print('For classifier:', classifier_type)

    for feature_key in all_features_for_classifier:
        print('  Training on:', feature_key)

        classifier['trained_classifiers'][feature_key] = {}

        curr_features = features[feature_key]['train_features'][cv_index]
        curr_labels = features[feature_key]['train_label'][cv_index]
        noise_features = None

        if classifier_type == 'fhmm':
            noise_features = features[feature_key]['noise_features'][cv_index]
            noise_features = np.array([item for sublist in noise_features for item in sublist])

        for speaker in sls['labels_set']:

            print('training for speaker: ', speaker)

            speaker_num = sls['label_to_num'][speaker]
            mask = (curr_labels == speaker_num)
            mask = mask.tolist()

            speaker_features = [array for array, flag in zip(curr_features, mask) if flag]
            speaker_features = np.array([item for sublist in speaker_features for item in sublist]) # TODO use lengths instead of flattening ?

            curr_classifier = None
            if classifier_type == 'fhmm':
                # FHMM
                curr_classifier = fhmm.FHMM(n_components_a=n_components, n_components_b=n_components)
                curr_classifier.fit(speaker_features, noise_features)
            elif classifier_type == 'GaussianHMM':
                # GAUSSIAN HMM
                curr_classifier = GaussianHMM(n_components=n_components, covariance_type='diag')
                curr_classifier.fit(speaker_features)
            elif classifier_type == 'GMMHMM':
                # GAUSSIAN HMM
                curr_classifier = GMMHMM(n_components=n_components, covariance_type='diag', n_mix=3)
                curr_classifier.fit(speaker_features)

            classifier['trained_classifiers'][feature_key][speaker] = curr_classifier

# use_fhmm = False
# print('training for feature type: ', feature_key)

# if feature_key.__contains__('Noise') and feature_key.__contains__('lp'):
#     print('using fhmm!')
#     use_fhmm = True

# classifiers[feature_key] = {}
# curr_features = features[feature_key]['train_features'][cv_index]
# curr_labels = features[feature_key]['train_label'][cv_index]

# if use_fhmm:
#     noise_features = features[feature_key]['noise_features'][cv_index]
#     noise_features = np.array([item for sublist in noise_features for item in sublist])
# print('score: ', curr_classifier.score(speaker_features))


For classifier: GaussianHMM
  Training on: lp
training for speaker:  F09
training for speaker:  M03
training for speaker:  F10
training for speaker:  M10
training for speaker:  F06
training for speaker:  F02
training for speaker:  F08
training for speaker:  M02
training for speaker:  M05
training for speaker:  F01
training for speaker:  M04
training for speaker:  M07
training for speaker:  M08
training for speaker:  F05
training for speaker:  M06
training for speaker:  M01
training for speaker:  F04
training for speaker:  F07
training for speaker:  F03
training for speaker:  M09
  Training on: lpGaussNoise_SNR60
training for speaker:  F09
training for speaker:  M03
training for speaker:  F10
training for speaker:  M10
training for speaker:  F06
training for speaker:  F02
training for speaker:  F08
training for speaker:  M02
training for speaker:  M05
training for speaker:  F01
training for speaker:  M04
training for speaker:  M07
training for speaker:  M08
training for speaker:  F05
tr

In [38]:
import pickle
pickle.dump(classifiers, open('classifiers.pickle','wb'))